# Flexx demo - Hijacking the notebook

In [ ]:
from flexx import app, event, ui
app.init_notebook()

### Create a dock panel and attach to ``<body>``

In [ ]:
dock = ui.DockPanel(container='body')

In [ ]:
from flexx.ui.examples.twente import Twente
twente = Twente(parent=dock, title='Temperature')

In [ ]:
from flexx.ui.examples.splines import Splines
splines = Splines(parent=dock, title='Splines')

Now the hijack ....

In [ ]:
class JupyterContainer(ui.Widget):
    
    class JS:
        
        def init(self):
            for name in ('header', 'site'):
                el = document.getElementById(name)
                self.node.appendChild(el)

In [ ]:
notebook = JupyterContainer(parent=dock, title='Notebook')

In [ ]:
splines.closed.checked = True

### Add Bokeh plot

In [ ]:
from bokeh.models import Range1d, WMTSTileSource, ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.airports import data as airports

title = "US Airports: Field Elevation > 1500m"

points_source = ColumnDataSource(airports)
tile_source = WMTSTileSource(url='http://otile2.mqcdn.com/tiles/1.0.0/sat/{Z}/{X}/{Y}.png')

x_range = Range1d(start=airports['x'].min() - 10000, end=airports['x'].max() + 10000, bounds=None)
y_range = Range1d(start=airports['y'].min() - 10000, end=airports['y'].max() + 10000, bounds=None)

p = figure(tools='wheel_zoom,pan', x_range=x_range, y_range=y_range, title=title)
p.axis.visible = False
hover_tool = HoverTool(tooltips=[("Name", "@name"), ("Elevation", "@elevation (m)")])
p.add_tools(hover_tool)
p.add_tile(tile_source)
p.circle(x='x', y='y', size=9, fill_color="#60ACA1", line_color="#D2C4C1", line_width=1.5, source=points_source)

In [ ]:
bw1 = ui.BokehWidget(plot=p, title='Airports', style='height:400px')
bw1

In [ ]:
bw1.parent = dock

In [ ]:
import numpy as np
from bokeh.plotting import figure 

N = 10000
x = np.random.normal(0, np.pi, N)
y = np.sin(x) + np.random.normal(0, 0.2, N)

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"
p = figure(tools=TOOLS, webgl=True)

p.circle(x, y, alpha=0.1, nonselection_alpha=0.01)

In [ ]:
bw2 = ui.BokehWidget(plot=p, title='10K points', style='height:400px')
bw2

In [ ]:
bw2.parent = dock